In [228]:
import pandas as pd
import json
import numpy as np
from pandasql import sqldf
import statistics 
import math
import scipy as stats
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import datetime
from datetime import  date, datetime, timedelta

In [229]:
# connect to database
con = sqlite3.connect('/Users/kaylalocklear/Documents/tableau/archive/sqlite-sakila.db')

In [230]:
# sql query
df = pd.read_sql_query('''select film.title ,
payment.amount as payment_amount,film.title ,
customer.first_name || " " || customer.last_name as customer_name,
rental.rental_date ,rental.return_date  ,  film.replacement_cost ,
staff.first_name, staff.first_name || staff.last_name as staff_name,
city.city , country.country, store.store_id 
from rental
left join inventory on inventory.inventory_id  = rental.inventory_id 
left join film on film.film_id = inventory.film_id 
left join customer on customer.customer_id = rental.customer_id 
left join payment on payment.rental_id = rental.rental_id  
left join staff on staff.staff_id = rental.staff_id   
left join address on address.address_id = customer.address_id 
left join store on store.manager_staff_id = staff.staff_id 
left join city on city.city_id = address.city_id 
left join country on country.country_id = city.country_id
 ''',con)
df

,title,payment_amount,title,customer_name,rental_date,return_date,replacement_cost,first_name,staff_name,city,country,store_id
0,BLANKET BEVERLY,2.99,BLANKET BEVERLY,CHARLOTTE HUNTER,2005-05-24 22:53:30.000,2005-05-26 22:04:30.000,21.99,Mike,MikeHillyer,guas Lindas de Gois,Brazil,1
1,FREAKY POCUS,2.99,FREAKY POCUS,TOMMY COLLAZO,2005-05-24 22:54:33.000,2005-05-28 19:40:33.000,16.99,Mike,MikeHillyer,Qomsheh,Iran,1
2,GRADUATE LORD,3.99,GRADUATE LORD,MANUEL MURRELL,2005-05-24 23:03:39.000,2005-06-01 22:12:39.000,14.99,Mike,MikeHillyer,Jaffna,Sri Lanka,1
3,LOVE SUICIDES,4.99,LOVE SUICIDES,ANDREW PURDY,2005-05-24 23:04:41.000,2005-06-03 01:43:41.000,21.99,Jon,JonStephens,Baku,Azerbaijan,2
4,IDOLS SNATCHERS,6.99,IDOLS SNATCHERS,DELORES HANSEN,2005-05-24 23:05:21.000,2005-06-02 04:33:21.000,29.99,Mike,MikeHillyer,Jaroslavl,Russian Federation,1
...,...,...,...,...,...,...,...,...,...,...,...,...
16039,COMANCHEROS ENEMY,0.99,COMANCHEROS ENEMY,BETTY WHITE,2005-08-23 22:25:26.000,2005-08-25 23:54:26.000,23.99,Mike,MikeHillyer,Citrus Heights,United States,1
16040,VOYAGE LEGALLY,0.99,VOYAGE LEGALLY,DENISE KELLY,2005-08-23 22:26:47.000,2005-08-27 18:02:47.000,28.99,Jon,JonStephens,Rio Claro,Brazil,2
16041,ILLUSION AMELIE,0.99,ILLUSION AMELIE,GRACE ELLIS,2005-08-23 22:42:48.000,2005-08-25 02:48:48.000,15.99,Jon,JonStephens,Duisburg,Germany,2
16042,HUNCHBACK IMPOSSIBLE,8.99,HUNCHBACK IMPOSSIBLE,GLADYS HAMILTON,2005-08-23 22:43:07.000,2005-08-31 21:33:07.000,28.99,Mike,MikeHillyer,Ilorin,Nigeria,1


In [231]:
# no duplicates
df.duplicated().sum()

0

In [232]:
# categorizing sales by rep - 4.2 is average for a sale 
df['col'] = df['payment_amount'].apply(lambda x: 'Above Avg' if x > 4.2
                                       else 'Avg' if x==4.2 else 'Below Avg'
                                       if x < 4.2 else 'No Sale')
    
df

# df['new column name'] = 
# df['column name'].apply(lambda x: 'value if condition is met' 
# if x condition else 'value if condition is not met')


,title,payment_amount,title,customer_name,rental_date,return_date,replacement_cost,first_name,staff_name,city,country,store_id,col
0,BLANKET BEVERLY,2.99,BLANKET BEVERLY,CHARLOTTE HUNTER,2005-05-24 22:53:30.000,2005-05-26 22:04:30.000,21.99,Mike,MikeHillyer,guas Lindas de Gois,Brazil,1,Below Avg
1,FREAKY POCUS,2.99,FREAKY POCUS,TOMMY COLLAZO,2005-05-24 22:54:33.000,2005-05-28 19:40:33.000,16.99,Mike,MikeHillyer,Qomsheh,Iran,1,Below Avg
2,GRADUATE LORD,3.99,GRADUATE LORD,MANUEL MURRELL,2005-05-24 23:03:39.000,2005-06-01 22:12:39.000,14.99,Mike,MikeHillyer,Jaffna,Sri Lanka,1,Below Avg
3,LOVE SUICIDES,4.99,LOVE SUICIDES,ANDREW PURDY,2005-05-24 23:04:41.000,2005-06-03 01:43:41.000,21.99,Jon,JonStephens,Baku,Azerbaijan,2,Above Avg
4,IDOLS SNATCHERS,6.99,IDOLS SNATCHERS,DELORES HANSEN,2005-05-24 23:05:21.000,2005-06-02 04:33:21.000,29.99,Mike,MikeHillyer,Jaroslavl,Russian Federation,1,Above Avg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,COMANCHEROS ENEMY,0.99,COMANCHEROS ENEMY,BETTY WHITE,2005-08-23 22:25:26.000,2005-08-25 23:54:26.000,23.99,Mike,MikeHillyer,Citrus Heights,United States,1,Below Avg
16040,VOYAGE LEGALLY,0.99,VOYAGE LEGALLY,DENISE KELLY,2005-08-23 22:26:47.000,2005-08-27 18:02:47.000,28.99,Jon,JonStephens,Rio Claro,Brazil,2,Below Avg
16041,ILLUSION AMELIE,0.99,ILLUSION AMELIE,GRACE ELLIS,2005-08-23 22:42:48.000,2005-08-25 02:48:48.000,15.99,Jon,JonStephens,Duisburg,Germany,2,Below Avg
16042,HUNCHBACK IMPOSSIBLE,8.99,HUNCHBACK IMPOSSIBLE,GLADYS HAMILTON,2005-08-23 22:43:07.000,2005-08-31 21:33:07.000,28.99,Mike,MikeHillyer,Ilorin,Nigeria,1,Above Avg


In [233]:
group = df.groupby(['staff_name','col'])['col'].count()
group.to_frame()

col
staff_name  col            
JonStephens Above Avg  3886
            Below Avg  4118
MikeHillyer Above Avg  3860
            Below Avg  4180

In [234]:
df.head(50)

,title,payment_amount,title,customer_name,rental_date,return_date,replacement_cost,first_name,staff_name,city,country,store_id,col
0,BLANKET BEVERLY,2.99,BLANKET BEVERLY,CHARLOTTE HUNTER,2005-05-24 22:53:30.000,2005-05-26 22:04:30.000,21.99,Mike,MikeHillyer,guas Lindas de Gois,Brazil,1,Below Avg
1,FREAKY POCUS,2.99,FREAKY POCUS,TOMMY COLLAZO,2005-05-24 22:54:33.000,2005-05-28 19:40:33.000,16.99,Mike,MikeHillyer,Qomsheh,Iran,1,Below Avg
2,GRADUATE LORD,3.99,GRADUATE LORD,MANUEL MURRELL,2005-05-24 23:03:39.000,2005-06-01 22:12:39.000,14.99,Mike,MikeHillyer,Jaffna,Sri Lanka,1,Below Avg
3,LOVE SUICIDES,4.99,LOVE SUICIDES,ANDREW PURDY,2005-05-24 23:04:41.000,2005-06-03 01:43:41.000,21.99,Jon,JonStephens,Baku,Azerbaijan,2,Above Avg
4,IDOLS SNATCHERS,6.99,IDOLS SNATCHERS,DELORES HANSEN,2005-05-24 23:05:21.000,2005-06-02 04:33:21.000,29.99,Mike,MikeHillyer,Jaroslavl,Russian Federation,1,Above Avg
5,MYSTIC TRUMAN,0.99,MYSTIC TRUMAN,NELSON CHRISTENSON,2005-05-24 23:08:07.000,2005-05-27 01:32:07.000,19.99,Mike,MikeHillyer,Santiago de Compostela,Spain,1,Below Avg
6,SWARM GOLD,1.99,SWARM GOLD,CASSANDRA WALTERS,2005-05-24 23:11:53.000,2005-05-29 20:34:53.000,12.99,Jon,JonStephens,Salinas,United States,2,Below Avg
7,LAWLESS VISION,4.99,LAWLESS VISION,MINNIE ROMERO,2005-05-24 23:31:46.000,2005-05-27 23:33:46.000,29.99,Jon,JonStephens,Ciomas,Indonesia,2,Above Avg
8,MATRIX SNOWMAN,4.99,MATRIX SNOWMAN,ELLEN SIMPSON,2005-05-25 00:00:40.000,2005-05-28 00:22:40.000,9.99,Mike,MikeHillyer,Po,Brazil,1,Above Avg
9,HANGING DEEP,5.99,HANGING DEEP,DANNY ISOM,2005-05-25 00:02:21.000,2005-05-31 22:44:21.000,18.99,Jon,JonStephens,Okara,Pakistan,2,Above Avg


In [235]:
df['payment_amount'].mean()

4.201356270256309

In [236]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16044 entries, 0 to 16043
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             16044 non-null  object 
 1   payment_amount    16044 non-null  float64
 2   title             16044 non-null  object 
 3   customer_name     16044 non-null  object 
 4   rental_date       16044 non-null  object 
 5   return_date       15861 non-null  object 
 6   replacement_cost  16044 non-null  float64
 7   first_name        16044 non-null  object 
 8   staff_name        16044 non-null  object 
 9   city              16044 non-null  object 
 10  country           16044 non-null  object 
 11  store_id          16044 non-null  int64  
 12  col               16044 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB


In [237]:
df['USA'] = df['country'].apply(lambda x: 'USA' if x == 'United States'
                                       else 'Other')
    
df

,title,payment_amount,title,customer_name,rental_date,return_date,replacement_cost,first_name,staff_name,city,country,store_id,col,USA
0,BLANKET BEVERLY,2.99,BLANKET BEVERLY,CHARLOTTE HUNTER,2005-05-24 22:53:30.000,2005-05-26 22:04:30.000,21.99,Mike,MikeHillyer,guas Lindas de Gois,Brazil,1,Below Avg,Other
1,FREAKY POCUS,2.99,FREAKY POCUS,TOMMY COLLAZO,2005-05-24 22:54:33.000,2005-05-28 19:40:33.000,16.99,Mike,MikeHillyer,Qomsheh,Iran,1,Below Avg,Other
2,GRADUATE LORD,3.99,GRADUATE LORD,MANUEL MURRELL,2005-05-24 23:03:39.000,2005-06-01 22:12:39.000,14.99,Mike,MikeHillyer,Jaffna,Sri Lanka,1,Below Avg,Other
3,LOVE SUICIDES,4.99,LOVE SUICIDES,ANDREW PURDY,2005-05-24 23:04:41.000,2005-06-03 01:43:41.000,21.99,Jon,JonStephens,Baku,Azerbaijan,2,Above Avg,Other
4,IDOLS SNATCHERS,6.99,IDOLS SNATCHERS,DELORES HANSEN,2005-05-24 23:05:21.000,2005-06-02 04:33:21.000,29.99,Mike,MikeHillyer,Jaroslavl,Russian Federation,1,Above Avg,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,COMANCHEROS ENEMY,0.99,COMANCHEROS ENEMY,BETTY WHITE,2005-08-23 22:25:26.000,2005-08-25 23:54:26.000,23.99,Mike,MikeHillyer,Citrus Heights,United States,1,Below Avg,USA
16040,VOYAGE LEGALLY,0.99,VOYAGE LEGALLY,DENISE KELLY,2005-08-23 22:26:47.000,2005-08-27 18:02:47.000,28.99,Jon,JonStephens,Rio Claro,Brazil,2,Below Avg,Other
16041,ILLUSION AMELIE,0.99,ILLUSION AMELIE,GRACE ELLIS,2005-08-23 22:42:48.000,2005-08-25 02:48:48.000,15.99,Jon,JonStephens,Duisburg,Germany,2,Below Avg,Other
16042,HUNCHBACK IMPOSSIBLE,8.99,HUNCHBACK IMPOSSIBLE,GLADYS HAMILTON,2005-08-23 22:43:07.000,2005-08-31 21:33:07.000,28.99,Mike,MikeHillyer,Ilorin,Nigeria,1,Above Avg,Other


In [238]:
# save as csv
df.to_csv('challenge2_df.csv', index = False)

In [211]:
# actors and film info
df2 = pd.read_sql_query('''select actor.first_name || " " || actor.last_name as actor_name, film.title , category.name,
"language".name, film.rating ,film.rental_rate, film.rental_duration 
from film
left join film_actor on film_actor.film_id = film.film_id 
left join actor on actor.actor_id = film_actor.actor_id
left join film_category on film_category.film_id = film.film_id
left join category on category.category_id = film_category.category_id 
left join "language" on "language".language_id = film.language_id''', con)
df2

,actor_name,title,name,name,rating,rental_rate,rental_duration
0,PENELOPE GUINESS,ACADEMY DINOSAUR,Documentary,English,PG,0.99,6
1,CHRISTIAN GABLE,ACADEMY DINOSAUR,Documentary,English,PG,0.99,6
2,LUCILLE TRACY,ACADEMY DINOSAUR,Documentary,English,PG,0.99,6
3,SANDRA PECK,ACADEMY DINOSAUR,Documentary,English,PG,0.99,6
4,JOHNNY CAGE,ACADEMY DINOSAUR,Documentary,English,PG,0.99,6
...,...,...,...,...,...,...,...
5460,WHOOPI HURT,ZOOLANDER FICTION,Children,English,R,2.99,5
5461,JADA RYDER,ZOOLANDER FICTION,Children,English,R,2.99,5
5462,IAN TANDY,ZORRO ARK,Comedy,English,NC-17,4.99,3
5463,NICK DEGENERES,ZORRO ARK,Comedy,English,NC-17,4.99,3


In [212]:
# close connection
con.close()

In [213]:
df2.duplicated().sum()

0

In [214]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5465 entries, 0 to 5464
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   actor_name       5462 non-null   object 
 1   title            5465 non-null   object 
 2   name             5465 non-null   object 
 3   name             5465 non-null   object 
 4   rating           5465 non-null   object 
 5   rental_rate      5465 non-null   float64
 6   rental_duration  5465 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 299.0+ KB


In [215]:
df2.isna().sum()

actor_name         3
title              0
name               0
name               0
rating             0
rental_rate        0
rental_duration    0
dtype: int64

In [216]:
df2.to_csv('challenge2_df2.csv', index = False)

In [222]:
df3 = pd.read_sql_query('''select film.title, country.country 
from rental
left join inventory on inventory.inventory_id  = rental.inventory_id 
left join film on film.film_id = inventory.film_id 
left join customer on customer.customer_id = rental.customer_id 
left join payment on payment.rental_id = rental.rental_id  
left join staff on staff.staff_id = rental.staff_id   
left join address on address.address_id = customer.address_id 
left join store on store.manager_staff_id = staff.staff_id 
left join city on city.city_id = address.city_id 
left join country on country.country_id = city.country_id''',con)
df3

,title,country
0,BLANKET BEVERLY,Brazil
1,FREAKY POCUS,Iran
2,GRADUATE LORD,Sri Lanka
3,LOVE SUICIDES,Azerbaijan
4,IDOLS SNATCHERS,Russian Federation
...,...,...
16039,COMANCHEROS ENEMY,United States
16040,VOYAGE LEGALLY,Brazil
16041,ILLUSION AMELIE,Germany
16042,HUNCHBACK IMPOSSIBLE,Nigeria


In [223]:
df3.to_csv('challenge2_df3.csv', index = False)

In [221]:
 group = df3.groupby(['country'])['title'].count()
group.to_frame()
group.head(50)

country
Afghanistan                                18
Algeria                                    90
American Samoa                             20
Angola                                     52
Anguilla                                   35
Argentina                                 352
Armenia                                    25
Austria                                    78
Azerbaijan                                 57
Bahrain                                    25
Bangladesh                                 95
Belarus                                    66
Bolivia                                    47
Brazil                                    748
Brunei                                     35
Bulgaria                                   50
Cambodia                                   53
Cameroon                                   54
Canada                                    137
Chad                                       32
Chile                                      71
China                     